In [ ]:
import pandas as pd
import requests
import joblib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = tf.keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = tf.keras.layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(res)
    x = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def create_transformer_model(df):
    inputs = tf.keras.layers.Input(shape=(None, ), dtype=tf.int32)
    x = tf.keras.layers.Embedding(input_dim=5000, output_dim=16)(inputs)
    x = transformer_encoder(x, head_size=256, num_heads=4, ff_dim=4*256, dropout=0.1)
    x = transformer_encoder(x, head_size=256, num_heads=4, ff_dim=4*256, dropout=0.1)
    x = transformer_encoder(x, head_size=256, num_heads=4, ff_dim=4*256, dropout=0.1)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    outputs = tf.keras.layers.Dense(len(df['internalStatus'].unique()), activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    return model

def preprocess_data(df):
    # Tokenize the 'externalStatus' column
    tokenizer = Tokenizer(num_words=5000, oov_token="OOV")
    tokenizer.fit_on_texts(df['externalStatus'])
    X = tokenizer.texts_to_sequences(df['externalStatus'])
    X = pad_sequences(X, padding='post')

    # Encode the 'internalStatus' column
    encoder = LabelEncoder()
    y = encoder.fit_transform(df['internalStatus'])

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Save tokenizer and encoder
    joblib.dump(tokenizer, 'tokenizer.joblib')
    joblib.dump(encoder, 'encoder.joblib')

    return X_train, X_test, y_train, y_test, tokenizer, encoder

def train_model(X_train, y_train, model):
    # Initialize the StratifiedKFold class
    skf = StratifiedKFold(n_splits=5)

    # Loop over each split
    for train_index, val_index in skf.split(X_train, y_train):
        # Split the data
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        # Compile the model
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        # Train the model with early stopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=3)
        history = model.fit(X_train_fold, y_train_fold, epochs=10, validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])

    return model

# Load the data
url = "https://gist.githubusercontent.com/farhaan-settyl/ecf9c1e7ab7374f18e4400b7a3d2a161/raw/f94652f217eeca83e36dab9d08727caf79ebdecf/dataset.json"
response = requests.get(url)
data = response.json()
df = pd.json_normalize(data)

# Preprocess the data
X_train, X_test, y_train, y_test, tokenizer, encoder = preprocess_data(df)

# Create the model
model = create_transformer_model(df)

# Train the model
model = train_model(X_train, y_train, model)

# Save the model
model.save('transformer_model.h5')

model.save_weights('transformer_weights.h5')

Epoch 1/10
25/25 [==============================] - 13s 221ms/step - loss: 2.2028 - accuracy: 0.3624 - val_loss: 1.3001 - val_accuracy: 0.6173
Epoch 2/10
25/25 [==============================] - 3s 128ms/step - loss: 0.7693 - accuracy: 0.8438 - val_loss: 0.3436 - val_accuracy: 0.9388
Epoch 3/10
25/25 [==============================] - 3s 129ms/step - loss: 0.2673 - accuracy: 0.9680 - val_loss: 0.2494 - val_accuracy: 0.9745
Epoch 4/10
25/25 [==============================] - 4s 163ms/step - loss: 0.1920 - accuracy: 0.9859 - val_loss: 0.2644 - val_accuracy: 0.9694
Epoch 5/10
25/25 [==============================] - 3s 123ms/step - loss: 0.1879 - accuracy: 0.9846 - val_loss: 0.2558 - val_accuracy: 0.9694
Epoch 6/10
25/25 [==============================] - 3s 126ms/step - loss: 0.1656 - accuracy: 0.9910 - val_loss: 0.2634 - val_accuracy: 0.9694
Epoch 1/10
25/25 [==============================] - 11s 144ms/step - loss: 0.2050 - accuracy: 0.9795 - val_loss: 0.1702 - val_accuracy: 0.9847
Epoc

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
num_classes = df['internalStatus'].nunique()
print(num_classes)

15


In [ ]:
print(tf.__version__)

2.15.0


In [ ]:
!pip show keras

Name: keras
Version: 2.15.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: tensorflow


In [ ]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: bigframes, fastai, imbalanced-learn, librosa, mlxtend, qudida, sklearn-pandas, yellowbrick


In [ ]:
!python --version

Python 3.10.12
